In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200

config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
            .set('spark.cores.max', '10')\
            .set('spark.port.maxRetries','200')\
            .set('spark.ui.port', port)\
            .set('spark.master', 'spark://polyp1:7077')\
            .set('spark.app.name', "structuredComputation")

#sc = pyspark.SparkContext(conf = config)
ss = pyspark.sql.SparkSession.builder.config(conf = config).getOrCreate()

# print ss.conf.get('spark.app.name')
# print ss.conf.get('spark.master')

In [3]:
#Task 2: Build Data Frame of the given data
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dates").getOrCreate()
df2 = spark.read.csv("/scratch/ISE495/taxi_data/green_tripdata_2015-09.csv",header = False,inferSchema = True)

In [4]:
df2.show()

+---+-------------------+-------------------+---+---+------------------+------------------+------------------+------------------+---+----+----+----+----+----+----+----+----+----+----+----+
|_c0|                _c1|                _c2|_c3|_c4|               _c5|               _c6|               _c7|               _c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|
+---+-------------------+-------------------+---+---+------------------+------------------+------------------+------------------+---+----+----+----+----+----+----+----+----+----+----+----+
|  2|2015-09-01 00:02:34|2015-09-01 00:02:38|  N|  5|-73.97948455810547| 40.68495559692383|-73.97943115234375|40.685020446777344|  1| 0.0| 7.8| 0.0| 0.0|1.95| 0.0|null| 0.0|9.75|   1|   2|
|  2|2015-09-01 00:04:20|2015-09-01 00:04:24|  N|  5|-74.01079559326172| 40.91221618652344|-74.01078033447266| 40.91221237182617|  1| 0.0|45.0| 0.0| 0.0| 0.0| 0.0|null| 0.0|45.0|   1|   2|
|  2|2015-09-01 00:01:50|2015-09-01 00:04:24|  N|  1| -

In [5]:
df2.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: timestamp (nullable = true)
 |-- _c2: timestamp (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)



In [6]:
# Change Header Names of the columns in Data Frame:
oldColumns = df2.schema.names
newColumns = ["VendorID","lpep_pickup_datetime","Lpep_dropoff_datetime","Store_and_fwd_flag","RateCodeID","Pickup_longitude","Pickup_latitude","Dropoff_longitude","Dropoff_latitude","Passenger_count","Trip_distance","Fare_amount","Extra","MTA_tax","Tip_amount","Tolls_amount","Ehail_fee","improvement_surcharge","Total_amount","Payment_type","Trip_type"]

df3 = reduce(lambda df2, idx: df2.withColumnRenamed(oldColumns[idx], newColumns[idx]), xrange(len(oldColumns)), df2)
df3.printSchema()
df3.show()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- Lpep_dropoff_datetime: timestamp (nullable = true)
 |-- Store_and_fwd_flag: string (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- Pickup_longitude: double (nullable = true)
 |-- Pickup_latitude: double (nullable = true)
 |-- Dropoff_longitude: double (nullable = true)
 |-- Dropoff_latitude: double (nullable = true)
 |-- Passenger_count: integer (nullable = true)
 |-- Trip_distance: double (nullable = true)
 |-- Fare_amount: double (nullable = true)
 |-- Extra: double (nullable = true)
 |-- MTA_tax: double (nullable = true)
 |-- Tip_amount: double (nullable = true)
 |-- Tolls_amount: double (nullable = true)
 |-- Ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- Total_amount: double (nullable = true)
 |-- Payment_type: integer (nullable = true)
 |-- Trip_type: integer (nullable = true)

+--------+--------------------+---

In [7]:
df3.createOrReplaceTempView("taxiTable")

In [8]:
# Task 2.1: Average Fare Amount by RateCodeID

jfk = ss.sql("SELECT DISTINCT RateCodeID FROM taxiTable")
print jfk.show()

+----------+
|RateCodeID|
+----------+
|         1|
|         6|
|         3|
|         5|
|         4|
|         2|
|        99|
+----------+

None


In [15]:
AvgFareAmtRateCodeID = df3.groupBy("RateCodeID").agg({'Fare_amount':'Average'})
print AvgFareAmtRateCodeID.show()

+----------+------------------+
|RateCodeID|  avg(Fare_amount)|
+----------+------------------+
|         1|12.244825220837368|
|         6| 2.138888888888889|
|         3| 48.79856759176365|
|         5|18.082432312995323|
|         4| 60.16488648648648|
|         2|49.021871476888386|
|        99|12.183333333333332|
+----------+------------------+

None


In [16]:
# Task 2.2: Airport Statistics:
     # Task 2.2.1: Number of rides per airport
     # Task 2.2.2: Footfall - jfk & newark - (number of passengers)
     # Task 2.2.3: Footfall - jfk & newark -  (Avg passenger count)
    # Task 2.2.4:  jfk & newark -  'passenger count' frequency by airport (trip counts for no. of passengers (eg. 1 passenger rides, 2 passenger rides)

In [17]:
# Task 2.2.1: Number of rides per airport
from pyspark.sql.functions import col, asc
df4 =  df3.filter((col("RateCodeID")=="2")|(col("RateCodeID")=="3")).groupBy("RateCodeID").agg({'RateCodeID':'count'})
print df4.show()

+----------+-----------------+
|RateCodeID|count(RateCodeID)|
+----------+-----------------+
|         3|             1117|
|         2|             4435|
+----------+-----------------+

None


In [18]:
# Task 2.2.2: Footfall - jfk & newark - (number of passengers)
df5 =  df3.filter((col("RateCodeID")=="2")|(col("RateCodeID")=="3")).groupBy("RateCodeID").agg({'Passenger_count':'sum'})
print df5.show()

+----------+--------------------+
|RateCodeID|sum(Passenger_count)|
+----------+--------------------+
|         3|                1560|
|         2|                5937|
+----------+--------------------+

None


In [20]:
# Task 2.2.3: Footfall - jfk & newark - (Avg of passengers)
df5 =  df3.filter((col("RateCodeID")=="2")|(col("RateCodeID")=="3")).groupBy("RateCodeID").agg({'Passenger_count':'average'})
print df5.show()

+----------+--------------------+
|RateCodeID|avg(Passenger_count)|
+----------+--------------------+
|         3|   1.396598030438675|
|         2|  1.3386696730552423|
+----------+--------------------+

None


In [21]:
# Task 2.2.4:  jfk  -  'passenger count' frequency by airport (trip counts for no. of passengers (eg. 1 passenger rides, 2 passenger rides)
jfkPASSCNT =  df3.filter(col("RateCodeID")=="2").groupBy("Passenger_count").agg({'Passenger_count':'count'})
jfkPASSCNT1 = jfkPASSCNT.sort("Passenger_count",ascending = True)
print jfkPASSCNT1.show()

+---------------+----------------------+
|Passenger_count|count(Passenger_count)|
+---------------+----------------------+
|              0|                     1|
|              1|                  3690|
|              2|                   450|
|              3|                    78|
|              4|                    19|
|              5|                   145|
|              6|                    52|
+---------------+----------------------+

None


In [22]:
# Task 2.2.4: newark(ewr) -  'passenger count' frequency by airport (trip counts for no. of passengers (eg. 1 passenger rides, 2 passenger rides)
ewrPASSCNT =  df3.filter(col("RateCodeID")=="3").groupBy("Passenger_count").agg({'Passenger_count':'count'})
ewrPASSCNT1 = ewrPASSCNT.sort("Passenger_count",ascending = True)
print ewrPASSCNT1.show()

+---------------+----------------------+
|Passenger_count|count(Passenger_count)|
+---------------+----------------------+
|              0|                     3|
|              1|                   902|
|              2|                   119|
|              3|                    26|
|              4|                     5|
|              5|                    50|
|              6|                    12|
+---------------+----------------------+

None


In [8]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format

In [11]:
#Average tip calculated for each day
df3v2 = df3.groupBy(dayofmonth(df3["lpep_pickup_datetime"])).agg({"Tip_amount":'average'})
df3v2.sort("dayofmonth(lpep_pickup_datetime)",ascending=True).show(31)

+--------------------------------+------------------+
|dayofmonth(lpep_pickup_datetime)|   avg(Tip_amount)|
+--------------------------------+------------------+
|                               1| 1.157276730921966|
|                               2|1.2257408345615415|
|                               3|1.2168314314066098|
|                               4|1.2084189511433254|
|                               5|1.1289811147563424|
|                               6|1.0957841236376002|
|                               7|1.1577241723506955|
|                               8| 1.195500255825858|
|                               9|1.2705117899822609|
|                              10|1.2496369941760872|
|                              11|1.2891907290141817|
|                              12|1.2688144021982883|
|                              13|1.2539606225456748|
|                              14|1.1721332176841344|
|                              15|1.1968545604742784|
|                           

In [12]:
#Average tip calculated for every hour 
tipHour = df3.groupBy(hour(df3["lpep_pickup_datetime"])).agg({"Total_amount":'average'})
tipHour.sort("hour(lpep_pickup_datetime)",ascending=True).show(24)

+--------------------------+------------------+
|hour(lpep_pickup_datetime)| avg(Total_amount)|
+--------------------------+------------------+
|                         0|15.052440066709117|
|                         1|14.505001022819105|
|                         2|14.482007476454758|
|                         3|14.839847661188882|
|                         4|15.723967983651521|
|                         5|17.715533532933957|
|                         6| 17.47073454802146|
|                         7|15.874489256277892|
|                         8|15.663039090987107|
|                         9|15.408813097522996|
|                        10| 14.89791275144532|
|                        11|14.755747565636481|
|                        12|14.706189908004136|
|                        13|14.606718339510806|
|                        14|14.732148235930708|
|                        15|14.892425417136774|
|                        16|15.425989868237759|
|                        17|15.007077435

In [13]:
#Total revenue calculated for each day 
RevenueDay = df3.groupBy(dayofmonth(df3["lpep_pickup_datetime"])).agg({"Total_amount":'sum'})
RevenueDay.sort("sum(Total_amount)",ascending=False).show(30)

+--------------------------------+------------------+
|dayofmonth(lpep_pickup_datetime)| sum(Total_amount)|
+--------------------------------+------------------+
|                              19|1059257.5600003442|
|                              12|1015901.0000008264|
|                              26| 994744.2600007713|
|                              18| 885568.3900005366|
|                               5| 882249.4200005603|
|                              11| 881832.7600005448|
|                              20|  879678.740000546|
|                              13| 838903.9000004525|
|                              27| 837646.7000004852|
|                               4| 825679.7299997478|
|                              10| 798203.1399997436|
|                               6| 789113.0100003557|
|                              25| 784947.7899997325|
|                              17| 761206.5600002861|
|                               3| 724530.8400001999|
|                           

In [14]:
ss.stop()

In [ ]:
sc.stop()